In [45]:
import numpy as np
import math
from itertools import product
#computes the entropy of a probability distribution given a list of probabilities "probabilities"
def H(probabilities):
    sum = 0.0
    for i in probabilities:
        if (i <= 0 and i>= -0.01):
            sum += 0
        else:
            sum -= i * math.log2(i)
    return sum

In [4]:
H([0.1, 0.9])

0.4689955935892812

In [335]:
#computes the objective function given numpy arrays p, a, b
def loss(p, a, b):
    n = len(np.atleast_1d(p))
    sum = 0
    for i in range(n):
        sum += p[i] * (H([a[i], 1 - a[i]]) + H([b[i], 1 - b[i]]))
    return sum

In [6]:
def h(x):
    return H([x, 1-x])

In [7]:
h(0.1)

0.4689955935892812

In [8]:
def L(x):
    return h(x) + 1 - x

In [336]:
# This computes the left hand side of the inequality in the conditions, given numpy arrays p, a, b, c (outdated)
def oldLHS(p, a, b, c):
    n = len(np.atleast_1d(p))
    sum1 = 0
    sum2 = 0
    sum3 = 0
    for i in range(n):
        sum1 += p[i] * (a[i] * b[i] - c[i])
        sum2 += p[i] * (a[i] * (1 - b[i]) + (1 - a[i]) * b[i] + 2 * c[i])
        sum3 += p[i] * ((1 - a[i]) * (1 - b[i]) - c[i])
    return H([sum1, sum2, sum3])

In [337]:
def LHS(p, a, b, c):
    n = len(np.atleast_1d(p))
    sum = 0
    for i in range(n):
        sum += (p[i] * (a[i] * (1 - b[i]) + b[i] * (1 - a[i]) + 2 * c[i]))
    if (sum < 0 or sum > 1):
        return False
    return L(sum)

In [353]:
# This computes the right hand side of the inequality in the conditions, given numpy arrays p, a, b, c

def RHS(p, a, b, c):
    entropy_sum = 0
    n = len(np.atleast_1d(p))
#     print(a)
#     print(b)
#     print(c)
    for i in range(n):
        if ((a[i] * b[i] - c[i]) < 0 or (a[i] * (1 - b[i]) + c[i]) < 0 or (b[i] * (1 - a[i]) + c[i]) < 0 or ((1 - a[i]) * (1 - b[i]) + c[i]) < 0):
            return False

    for i in range(n):
        entropy = H([a[i] * b[i] - c[i], a[i] * (1 - b[i]) + c[i], (1 - a[i]) * b[i] + c[i], (1 - a[i]) * (1 - b[i]) - c[i]])
        entropy_sum += p[i] * entropy
    return entropy_sum

In [339]:
# This computes quantity to minimize for each value of c, given numpy arrays p, a, b, c

def LHS_minus_RHS(p, a, b, c):
    if (LHS(p,a,b,c) == False or RHS(p,a,b,c) == False):
        return False
    return LHS(p, a, b, c) - RHS(p, a, b, c)

In [451]:
#Uses gradient descent to find the worst values of c possible

def worst_c(p, a, b, alpha, iterations):
    n = len(np.atleast_1d(p))
    c = np.zeros(n, dtype = float)
    for i in range(iterations):
        for j in range(n):
            cplus = list(c)
            cplus[j] += 0.0001
            cminus = list(c)
            cminus[j] -=  0.0001
            dxdc = (LHS_minus_RHS(p, a, b, cplus) - LHS_minus_RHS(p, a, b, cminus))/.0002
            c[j] -= dxdc * alpha
        for j in range(n):
            if (a[j] * b[j] - c[j] < 0 or a[j] * (1 - b[j]) + c[j] < 0 or b[j] * (1 - a[j]) + c[j] < 0 or (1 - a[j]) * (1 - b[j]) + c[j] < 0):
                c += dxdc * alpha
                return c
            
    return c

In [293]:
%%time
p = np.array((1.0/3, 1.0/3, 1.0/3))
atest = np.array((0.2, 0.5, 0.2))
btest = np.array((0.3, 0.3, 0.4))
c = worst_c(p, atest, btest, alpha = 0.01, iterations = 10)
# c = np.array((1,1,1))
print(LHS_minus_RHS(p, atest, btest, c))
print(c)

-0.18787224446955308
[-0.00598564  0.03217907  0.0057064 ]
CPU times: user 11.5 ms, sys: 795 µs, total: 12.2 ms
Wall time: 11.5 ms


In [267]:
%%time
p = np.array((0.05, 0.05, 0.9))
atest = np.array((0.2, 0.5, 0.2))
btest = np.array((0.3, 0.3, 0.4))
c = worst_c(p, atest, btest, alpha = 0.001, iterations = 1000)
# c = np.array((1,1,1))
print(LHS_minus_RHS(p, atest, btest, c))
print(c)

-0.1504782038516299
[-0.00624847  0.03743791  0.00728268]
CPU times: user 627 ms, sys: 4.07 ms, total: 631 ms
Wall time: 637 ms


In [453]:
# Ignore for now
a = np.array((0.2, 0.3, 0.4))
b = np.array((0.3, 0.4, 0.2))
p = np.array((0.3, 0.3, 0.4))
c = worst_c(p, a, b, alpha = 0.001, iterations = 10000)
print(c)
sumLHSnum = 0
cplus = list(c)
cplus[0] += 0.001
cminus = list(c)
cminus[0] -= 0.001
dxdc = (LHS(p, a, b, cplus) - LHS(p, a, b, cminus))/0.002
for i in range(3):
    sumLHSnum += (p[i] * (a[i] * (1 - b[i]) + b[i] * (1 - a[i]) + 2 * c[i]))
LHSval = math.pow((1 - sumLHSnum) / (2 * sumLHSnum), 2)
# print(sumLHS1)
# print(sumLHS2)
# print(sumLHS3)
x = 0
print(LHSval)
# print(math.log2(LHSval)*p[x])
# print(2 * p[x] * math.log2(LHSval))
RHSval = (a[x] * b[x] - c[x]) * ((1 - a[x]) * (1 - b[x]) - c[x])/(((1 - a[x]) * b[x] + c[x]) * ((1 - b[x]) * a[x] + c[x]))
print(RHSval)
# print(math.log2(RHSval) * p[x])
cplus[0] = 0.045
print(LHS_minus_RHS(p, a, b, c))

[0.04188441 0.0729197  0.05248741]
0.18306713227624133
0.18306804980881444
-0.19549993771628693


In [ ]:
# %%time
# #DO NOT RUN THIS YET, EVERYTHING ELSE NEEDS TO BE FIXED FIRST

x = np.linspace(0.0, 1, num = 11)
ptest = np.array((1.0/3, 1.0/3, 1.0/3))
maxloss = 0.0
pbest = 0
abest = 0
bbest = 0
cbest = 0

acheck = 0
bcheck = 0

for (a1, a2, a3, b1, b2, b3) in product(x, x, x, x, x, x):
    a = [a1, a2, a3]
    b = [b1, b2, b3]
    c = worst_c(ptest, a, b, alpha = 0.01, iterations = 10)
    for (p_1, p_2) in product(x, x):
        p_3 = 1 - p_1 - p_2
        if (p_3 < 0):
            continue
        p = [p_1, p_2, p_3]
        if LHS_minus_RHS(p, a, b, c) >= 0:
            if (maxloss < loss(p, a, b)):
                c = worst_c(p, a, b, alpha = 0.001, iterations = 1000)
                if LHS_minus_RHS(p, a, b, c) >= 0:
                    pbest = p
                    abest = a
                    bbest = b
                    cbest = c
                    maxloss = loss(p, a, b)
print(maxloss)

In [360]:
print(pbest)
print(abest)
print(bbest)
print(cbest)

[0.1, 0.4, 0.5]
[0.1, 0.2, 0.30000000000000004]
[0.2, 0.2, 0.30000000000000004]
[0.00709884 0.01305977 0.02385208]


In [311]:

c = worst_c(pbest, abest, cbest, alpha = 0.001, iterations = 1000)
print(c)
print(LHS_minus_RHS(pbest, abest, bbest, cbest))

[0. 0. 0.]
4.82001024346701e-05


In [357]:
a = np.array((0.23684, 0.23684, 0.23684))
b = np.array((0.23684, 0.23684, 0.23684))
c = np.array((0.0153821919, 0.0153821919, 0.0153821919))
p = np.array((0.2, 0.8))
print(worst_c(p, a, b, alpha = 0.001, iterations = 1000))

[0.01537418 0.01537666]


In [356]:
LHS_minus_RHS(p, a, b, worst_c(p, a, b, alpha = 0.001, iterations = 1000))

-8.511719146397922e-06

In [355]:
LHS_minus_RHS(p, a, b, c)

-8.512262101856294e-06

In [350]:
LHS(p, a, b, c)

1.5739832616659277

In [354]:
RHS(p, a, b, c)

1.5739917739280296

In [73]:
def star(p,a,b,c):
    n = len(np.atleast_1d(p))
    sum = 0
    for i in range(n):
        sum += (p[i] * (a[i] * (1 - b[i]) + b[i] * (1 - a[i]) + 2 * c[i]))
    if (sum < 0 or sum > 1):
        return False
    return sum
#LHS of the partial derivative expression
def delL(p,a,b,c):
    s = star(p,a,b,c)
    return ((1-s)/(2*s))**2

def solve_quad(a,b,c, low, high):
    s1 = (-1*b + (b**2-4*a*c)**.5)/(2*a)
    s2 = (-1*b - (b**2-4*a*c)**.5)/(2*a)
    if s1>=low and s1<=high:
        return s1
    if s2>=low and s2<=high:
        return s2
    return False

#finds worse c by solving for where partial derivative = 0 given old LHS
def update_c(s,pi,ai,bi):
    w = (1-ai)*bi
    x = ai * (1-bi)
    y = ai*bi
    z = (1-ai)*(1-bi)
    quad = s-1
    lin = s*w+s*x+y+z
    con = s*w*x - y*z
    return solve_quad(quad,lin,con, low = 0, high = 1)
def worse_c(p,a,b,c):
    s = delL(p,a,b,c)
    n = len(np.atleast_1d(p))
    c2 = np.zeros(n, dtype = float)
    for i in range(n):
        u = update_c(s,p[i],a[i],b[i])
        if u>=0 and u<=1:
            c2[i]=u
        else:
            c2[i]=c[i]
    return c2

In [74]:
a = np.array((0.23684, 0.23684, 0.23684))
b = np.array((0.23684, 0.23684, 0.23684))
c = np.array((0, 0, 0))
p = np.array((0.2, 0.8))
#starting at 0 vector and iterating a bunch could give the worst c vector
for i in range(100):
    c=worse_c(p,a,b,c)
print(c)
print(LHS_minus_RHS(p,a,b,c))

[0.01538219 0.01538219]
-8.51226210163425e-06


In [16]:
def check_c(p, a, b, c):
    n = len(np.atleast_1d(p))
    sum = 0
    for i in range(n):
        sum += p[i]*(a[i]*(1 - b[i]) + b[i]*(1 - a[i]) + 2*c[i])
    return sum

In [84]:
#possible problems: Need to check both roots of quadratic, accuracy needs to work
#Error can be lessened by decreasing step size of x
def ceiling_worst_c(p, a, b):
    n = len(p)
    c_worst = 0
    mindistance = 10000
    for x in  np.linspace(0.001, 1, num = 1000):
        c = []
        y = math.pow((1 - x)/(2 * x),2)
        #consider a quadratic equation pc^2+qc+r=0
        for i in range(n):
            pp = y - 1
            qq = 2*a[i]*b[i] - 2*a[i]*b[i]*y + a[i]*y + b[i]*y - a[i] - b[i] + 1
            rr = a[i]*a[i]*b[i]*b[i]*y - a[i]*a[i]*b[i]*b[i] - a[i]*a[i]*b[i]*y - a[i]*b[i]*b[i]*y + a[i]*a[i]*b[i] + a[i]*b[i]*b[i] + a[i]*b[i]*y - a[i]*b[i]
            if (solve_quad(pp, qq, rr, low = -0.25, high = 0.25) != False):
                #be careful, maybe the second root works. Need to check this never actually happens
                c.append(solve_quad(pp, qq, rr, low = -0.25, high = 0.25))
        if (len(c) != 3):
            continue
        if (abs(check_c(p, a, b, c) - x) < mindistance):
            mindistance = abs(check_c(p, a, b, c) - x)
            c_worst = c
    return c_worst
                
        
    

In [85]:
p = np.array((0.3, 0.3, 0.4))
a = np.array((0.23684, 0.23684, 0.23684))
b = np.array((0.23684, 0.23684, 0.23684))
print(worst_c(p, a, b, alpha = 0.0001, iterations = 10000))
print(ceiling_worst_c(p, a, b))

[0.01537563 0.01537563 0.01537611]
[0.015322893429498519, 0.015322893429498519, 0.015322893429498519]
